In [4]:
from functools import reduce
from json import loads

import pandas as pd
import requests
from fake_useragent import UserAgent

In [14]:
ua = UserAgent()
url = "https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-12-2024-15"
headers = {"User-Agent": ua.random}

response = requests.get(url, headers=headers)
startidx = response.text.find("(")
endidx = response.text.find(")")
data = loads(response.text[startidx + 1 : endidx])
series_list = loads(data["json"])

In [15]:
df_list = []
for series in series_list:
    df_dict = {}
    region = series["label"]
    df_dict["Timestamp"] = [
        pd.to_datetime(x[0], unit="ms") for x in series["data"]
    ]
    df_dict[region] = [int(x[1]) for x in series["data"]]
    df_list.append(pd.DataFrame(df_dict))

df = reduce(
    lambda x, y: pd.merge(x, y, on="Timestamp", how="outer"), df_list
)
df = df.sort_values("Timestamp").reset_index(drop=True)

In [16]:
df

,Timestamp,Central America,Africa,Middle East,Oceania,South America,Russia,Asia,Europe,North America
0,2025-01-02 00:20:00,143,77,589,409,2916,1118,2298,4892,10388
1,2025-01-02 00:30:00,142,74,558,409,2903,1085,2298,4694,10381
2,2025-01-02 00:40:00,142,74,558,411,2905,1085,2444,4694,10388
3,2025-01-02 00:50:00,142,72,547,426,2903,1044,2679,4434,10404
4,2025-01-02 01:00:00,145,72,524,437,2877,1003,2859,4184,10419
...,...,...,...,...,...,...,...,...,...,...
283,2025-01-03 23:30:00,115,78,662,296,2571,1140,1467,6122,7490
284,2025-01-03 23:40:00,120,78,642,313,2563,1098,1514,5847,7514
285,2025-01-03 23:50:00,121,78,638,329,2571,1069,1570,5627,7630
286,2025-01-04 00:00:00,121,77,622,335,2608,1035,1662,5419,7729
